<a href="https://colab.research.google.com/github/Ansh-1019/Mental-Health-Analysis-NLP/blob/main/Mental_Health_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers data

  Preparing metadata (setup.py) ... done
  Created wheel for data: filename=data-0.4-py3-none-any.whl size=7226 sha256=1d9af19d0f9bad3da1e620846b018010a9e06ef4dff35efce65afe2e869d3c3e
  Stored in directory: /root/.cache/pip/wheels/0e/90/87/0e55006f5c69af278687927041e334cfb928ab0a7ad9b0c1a3
Successfully built data


In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 137.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [ ]:
!pip install transformers==4.30.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.9/314.9 kB 17.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 79.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)


In [ ]:
!pip install matplotlib-venn

In [ ]:
!apt-get -qq install -y libarchive-dev && pip install -U libarchive

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 126435 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1.5_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Setting up libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Processing triggers for man-db (2.10.2-1) ...
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 5.6 MB/s eta 0:00:00
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31629 sha256=f759598012cc9b57389495f026dedd16ec73e8bc35895cc5e37ddd2c1776b4b1
  Stored in directory: /root/.cache/pip/wheels/29/20/ab/f101da7b245b996aa097685ef742243725ea6150f5b3b6d9ed
Successfully built libarchive


In [ ]:
!apt-get -qq install -y graphviz && pip install pydot

In [ ]:
!pip install cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 114.4 MB/s eta 0:00:00


In [ ]:
!pip install transformers accelerate datasets scikit-learn

  Using cached transformers-4.56.2-py3-none-any.whl.metadata (40 kB)
Using cached transformers-4.56.2-py3-none-any.whl (11.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 12.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Step 1: Install and Import Necessary Libraries
# Run these lines in a Colab cell first by removing the '#'
# !pip install transformers datasets accelerate scikit-learn
import libarchive
import pydot
import cartopy
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from huggingface_hub import notebook_login

def main():
    # Step 2: Log in to Hugging Face Hub
    print("Attempting to log in to Hugging Face Hub...")
    notebook_login()
    print("Hugging Face Hub login process initiated. Please follow the instructions.")


    # Step 3: Load Your Final 5-Class Dataset
    print("Loading the final 5-class balanced dataset...")
    df = pd.read_csv('final_balanced_5class_dataset.csv', encoding='latin-1')

    df['text'] = df['text'].fillna('')  # Replace empty cells (NaN) with an empty string
    df['text'] = df['text'].astype(str) # Ensure all values in the column are strings

    # Ensure the label column is named 'label' for the trainer
    df = df.rename(columns={'advanced_label': 'label'})
    dataset = Dataset.from_pandas(df)

    # Step 4: Split the Dataset
    train_test_split = dataset.train_test_split(test_size=0.2, seed=42)
    train_dataset = train_test_split['train']
    test_dataset = train_test_split['test']

    # Step 5: Tokenize the Data
    print("Tokenizing the dataset...")
    model_name = "distilbert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def tokenize_function(examples):
        return tokenizer(examples['text'], padding="max_length", truncation=True)

    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

    # Step 6: Load the Model (Configured for 5 Labels)
    print("Loading the pre-trained model for 5-class classification...")
    id2label = {0: "Depression", 1: "Anxiety", 2: "Suicidal Ideation", 3: "Happy", 4: "Neutral/Casual"}
    label2id = {"Depression": 0, "Anxiety": 1, "Suicidal Ideation": 2, "Happy": 3, "Neutral/Casual": 4}

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=5,  # <-- Crucial change for 5 classes
        id2label=id2label,
        label2id=label2id
    )

    # Step 7: Define Evaluation Metrics
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return {
            'accuracy': accuracy_score(labels, predictions),
            'f1': f1_score(labels, predictions, average='weighted'),
            'precision': precision_score(labels, predictions, average='weighted'),
            'recall': recall_score(labels, predictions, average='weighted'),
        }

    # Step 8: Define Training Arguments
    training_args = TrainingArguments(
        output_dir="Mental-Health-Analysis",
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        eval_strategy="steps", # Changed from epoch to steps
        eval_steps=500, # Added eval_steps
        save_steps=500,
        load_best_model_at_end=True,
        push_to_hub=True,
        hub_model_id="vedabtpatil07/Mental-Health-Analysis", # New model name
    )

    # Step 9: Create and Run the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_test_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

    print("🚀 Starting model training...")
    trainer.train()
    print("Training complete.")

    # # Step 10: Push the final model to the Hub
    # print("Pushing the model to the Huging Face Hub...")
    # trainer.push_to_hub()
    # print("Model successfully pushed to the Hub.")

if __name__ == "__main__":
    main()

AttributeError: partially initialized module 'torch' has no attribute 'types' (most likely due to a circular import)